In [76]:
import numpy as np

We note that we are using the base field `QQ` for all computations currently

In [77]:
# Let us initialize constants and vector spaces.

N = 16
n = 4
d = 4
indeterminates = [var("y_%d%d" % ((i+1),(j+1))) for i in range(n) for j in range(n)]
R = PolynomialRing(QQ, indeterminates)
R.inject_variables()

dimV = binomial(N + d - 1, d)
V = VectorSpace(QQ, dimV, sparse=True)
glN = MatrixSpace(QQ, N)

Defining y_11, y_12, y_13, y_14, y_21, y_22, y_23, y_24, y_31, y_32, y_33, y_34, y_41, y_42, y_43, y_44


Note $\hat{\sigma_{r}}(\mathbb{P}^{n-1} \times \mathbb{P}^{n-1}) = \mathbb{V}( (r+1)\times(r+1) \text{ minors})$

In [87]:
# Let us inititalize the list of polynomials, f
# In our case, r=3.

M = MatrixSpace(R, n)
A = M(np.array(indeterminates).reshape(-1,n))
print(A)
if n <= 3:
    f = [R(A.determinant())]
else:
    f = A.minors(4)
    
print(f)
    
grad_f = [fi.gradient() for fi in f]

[y_11 y_12 y_13 y_14]
[y_21 y_22 y_23 y_24]
[y_31 y_32 y_33 y_34]
[y_41 y_42 y_43 y_44]
[y_14*y_23*y_32*y_41 - y_13*y_24*y_32*y_41 - y_14*y_22*y_33*y_41 + y_12*y_24*y_33*y_41 + y_13*y_22*y_34*y_41 - y_12*y_23*y_34*y_41 - y_14*y_23*y_31*y_42 + y_13*y_24*y_31*y_42 + y_14*y_21*y_33*y_42 - y_11*y_24*y_33*y_42 - y_13*y_21*y_34*y_42 + y_11*y_23*y_34*y_42 + y_14*y_22*y_31*y_43 - y_12*y_24*y_31*y_43 - y_14*y_21*y_32*y_43 + y_11*y_24*y_32*y_43 + y_12*y_21*y_34*y_43 - y_11*y_22*y_34*y_43 - y_13*y_22*y_31*y_44 + y_12*y_23*y_31*y_44 + y_13*y_21*y_32*y_44 - y_11*y_23*y_32*y_44 - y_12*y_21*y_33*y_44 + y_11*y_22*y_33*y_44]


In [79]:
# This defines the action of glN (general linear lie algebra) on the homogeneous polynomials of degree d in R.

# X_action: (glN, R^d) => R^d
def X_action(X, i):
    sum = 0
    for a in range(N):
        for b in range(N):
            sum += -X[a][b]*indeterminates[b]*grad_f[i][a]
    return R(sum)

In [80]:
# This cell defines the explicit isomorphism between V <=> R^d

# construct monomial list
i = 0
elem = 0
lst = []
while i < N*d:
    lst.append(indeterminates[elem])
    i += 1
    if i % d == 0:
        elem += 1
S = Subsets(lst, d, submultiset=True).list()
monomials = [R(np.prod(S[i])) for i in range(dimV)]

# R^d => V
def polynomial_to_vector(p):
    if not p in R:
        return 0
    coefficient_list = []
    for i in range(len(monomials)):
        coefficient_list.append(p.monomial_coefficient(monomials[i]))
    return V(coefficient_list)

# V => R^d
def vector_to_polynomial(v):
    if not v in V:
        return 0
    p = 0
    for i, item in enumerate(Sequence(monomials)):
        p += v[i]*item
    return R(p)

In [81]:
# Data processing cell to flatten sparse vectors over V. We also create an optimized inner product function on such vectors.

# V => V
def sparsify(v):
    flattened_list = []
    for index, value in enumerate(v):
        if v[index] != 0:
            flattened_list.append((index,value))
    return flattened_list


# <V,V> => QQ
def sparse_inner_product(a,b):
    i = 0; j = 0
    result = 0
    while i < len(a) and j < len(b):
        if a[i][0] == b[j][0]:
            result += a[i][1]*b[j][1]
            i += 1
            j += 1
        elif a[i][0] < b[j][0]:
            i += 1
        else:
            j += 1
    return result

In [82]:
# W = span{f}

f_vec = [polynomial_to_vector(fi) for fi in f]
W = V.subspace(f_vec)

print(W)
p = W.dimension()

Sparse vector space of degree 3876 and dimension 1 over Rational Field
Basis matrix:
1 x 3876 sparse matrix over Rational Field


In [83]:

W_perp = W.complement()

print(W_perp)

g = W_perp.basis()
q = W_perp.dimension()

Vector space of degree 3876 and dimension 3875 over Rational Field
Basis matrix:
3875 x 3876 dense matrix over Rational Field


We note `M_f` $:= \tilde{M}_f$

In [84]:
# Pre-processing memoization of entries of M_f

# returns E_ij basis matrix of glN
E = lambda i,j: list(glN.basis())[(N*i) + j]

# E_action_f[l][i][j] = (E_ij)(f_l)
E_action_f = [[[sparsify(polynomial_to_vector(X_action(E(i,j),l))) for i in range(N)] for j in range(N)] for l in range(p)]

# convert g to sparse representation
g_sparse = [sparsify(g[i]) for i in range(q)]

In [85]:

# Construct M_f
M_f = zero_matrix(QQ, p*q, N^2)
for l in range(p):
    for k in range(q):
        for i in range(N):
            for j in range(N):
                M_f[(p*l) + k,(N*i) + j] = sparse_inner_product(E_action_f[l][i][j], g_sparse[k])

print('dim(ker(M_f)) =', glN.dimension() - M_f.rank())
print('2*(n^2)-1 =', 2*(N)-1)

dim(ker(M_f)) = 31
2*(n^2)-1 = 31
